<a href="https://colab.research.google.com/github/Lukehsu1999/PoetBot/blob/main/Classify_Poems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Import pretrained emotionClassifier </h1> 

Distribution of data based on labels:  <br>

joy    6761 <br>
sadness    5797 <br>
anger    2709 <br>
fear    2373 <br>
love    1641 <br>
surprise     719 <br>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.4MB 6.0MB/s 
     |████████████████████████████████| 890kB 40.0MB/s 
     |████████████████████████████████| 2.9MB 35.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d17e2cc0d7ee2e9c1e109c00baf12cbb2b062e2e2e886fb225ba42049711ce77
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertForSequenceClassification

emotionClassifier=BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/EmotionNLP/pretrainedModel", return_dict=True)


In [3]:
def classify_emotion(sentence):

  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
  input = tokenizer(sentence, return_tensors="pt")
  emotionClassifier.eval()
  labels=torch.tensor([1]).unsqueeze(0)

  outputs = emotionClassifier(**input,labels=labels)
  loss = outputs.loss
  logits = outputs.logits
  logits=logits.detach().numpy()
  probs = tf.nn.softmax(logits)
  prediction = tf.math.argmax(probs, axis=1)

  # 0:"anger", 1:"fear", 2:"joy", 3:"love", 4:"sadness", 5:"surprise",
  prediction=tf.constant(prediction)

  #print(probs)
  probs=tf.constant(probs)
  prob_prediction=probs.numpy()[0][prediction]
  prob_prediction=tf.constant(prob_prediction)
  #print(prediction)
  #print(prob_prediction)

  #print("prediction: "+str(prediction.numpy()[0]) +" with probability(certainty): "+str(prob_prediction.numpy()))
  
  #if probability(certainty) is bigger than 80%
  if prob_prediction>0.8:
    return prediction.numpy()[0]
  else:
    return -1


In [4]:
testsent="I am sad"
print(str(classify_emotion(testsent)))


4


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [5]:
index2emotion={
    0:"anger", 1:"fear", 2:"joy", 3:"love", 4:"sadness", 5:"surprise"
}

<h1>Get the Poetry dataset</h1>

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import spatial
from collections import Counter

import keras.backend as K
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, Input, LSTM, GRU

import matplotlib.pyplot as plt

In [7]:
poems_df = pd.read_csv("/content/drive/MyDrive/PoemDataset/kaggle_poem_dataset.csv")

In [8]:
len(poems_df)

15652

<h1> Classify the poems by Emotion <h1>

In [ ]:
anger_set=[]
fear_set=[]
joy_set=[]
love_set=[]
sadness_set=[]
surprise_set=[]

for i in range(0,1000):
  #print(i)
  #print(poems_df.iloc[i]["Author"])
  #print(poems_df.iloc[i]["Title"])
  #print(poems_df.iloc[i]["Content"])
  content=poems_df.iloc[i]["Content"]

  #Bert has max length 512
  if len(content)>512:
    content=content[0:512]

  emotion_idx=classify_emotion(content)
  if emotion_idx==0:
    anger_set.append(poems_df.iloc[i])
  elif emotion_idx==1:
    fear_set.append(poems_df.iloc[i])
  elif emotion_idx==2:
    joy_set.append(poems_df.iloc[i])
  elif emotion_idx==3:
    love_set.append(poems_df.iloc[i])
  elif emotion_idx==4:
    sadness_set.append(poems_df.iloc[i])
  elif emotion_idx==5:
    surprise_set.append(poems_df.iloc[i])
  #if emotion_idx=-1 -> the certainty of the result is lower than 80%


  #print("Classification Result: "+str(classify_emotion(content)))
  #print("##################################")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [ ]:
print(str(len(anger_set))+";"+str(len(fear_set))+";"+str(len(joy_set))+";"+str(len(love_set))+";"+str(len(sadness_set))+";"+str(len(surprise_set))+";")


In [ ]:
def printpoemset(poemset):
  for poem in poemset:
    print("Author: "+poem["Author"])
    print("Title: "+poem["Title"])
    print(poem["Content"])
    print()

<h1>saving results </h1>

txt approach

csv approach

In [ ]:
import csv

def save_result_csv(path,poemset):
  with open(path,'w',newline='') as csvfile:
    fieldnames=['Id','Author','Title','Content']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    cnt=0
    for poem in poemset:
      cnt=cnt+1
      writer.writerow({"Id":cnt,"Author":poem["Author"],"Title":poem["Title"],"Content":poem["Content"]})

In [ ]:
save_result_csv("/content/drive/MyDrive/EmotionPoem/joy_poems.csv",joy_set)
save_result_csv("/content/drive/MyDrive/EmotionPoem/sadness_poems.csv",sadness_set)
save_result_csv("/content/drive/MyDrive/EmotionPoem/anger_poems.csv",anger_set)
save_result_csv("/content/drive/MyDrive/EmotionPoem/fear_poems.csv",fear_set)
save_result_csv("/content/drive/MyDrive/EmotionPoem/love_poems.csv",love_set)
save_result_csv("/content/drive/MyDrive/EmotionPoem/surprise_poems.csv",surprise_set)